In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
def get_data():
    """Assumes the tki-resistance.csv is in our working directory.
    Returns (X_train, y_train, X_test, y_test)
    """
    data = pd.read_csv("tki-resistance.csv")
    data["Class"] = data["Class"].map({"Bcr-abl":0, "Wild type":1})
    X, y = np.array(data)[:,0:-1], np.array(data)[:,-1]
    X_train, y_train, X_test, y_test = X[:130], y[:130], X[130:], y[130:]

    return ((X_train, y_train), (X_test, y_test))

In [3]:
train, test = get_data()

In [14]:
def all_columns(X, rand):
    return range(X.shape[1])


def random_sqrt_columns(X, rand):
    c = rand.sample(range(0, X.shape[1]), round(X.shape[1]**0.5))
    return c


class Tree:

    def __init__(self, rand=None,
                 get_candidate_columns=all_columns,
                 min_samples=2):
        self.rand = rand  # for replicability
        self.get_candidate_columns = get_candidate_columns  # needed for random forests
        self.min_samples = min_samples 
    
    def build(self, X: np.array, y: np.array):
        """
        Recusrively build a tree, stop recursion when a split has a child node with gini impurity 0 or
        when we have less than min_samples samples.
        """
        if (len(y) == 0): # for an empty branch just return 0
            return TreeNode(None, None, 0)
        if (len(y) < self.min_samples): # we are in a leaf node
            return TreeNode(None, None, round(np.mean(y))) # make the majority class the prediction for this node
        if (np.all(y == 1)): # check if we have a node with all ones
            return TreeNode(None, None, 1)
        if (np.all(y == 0)):
            return TreeNode(None, None, 0)
        
        decision_rule = self.find_decision_rule(X, y)
        feature, split_value = decision_rule
        
        left_i = np.where(X[:,feature] < split_value)
        right_i = np.where(X[:, feature] >= split_value)
        
        left_subtree = Tree()
        right_subtree = Tree()
        
        return TreeNode(left_subtree.build(X[left_i], y[left_i]),
                        right_subtree.build(X[right_i], y[right_i]), 
                        decision_rule) 
    
    
    def find_decision_rule(self, X, y):
        """
        Input: X - data, y - labels
        Output: A tuple (left, right, decision_rule), left indicies, right indicies and rule. Rule itself is a tuple
        of the index of the feature to split on and the value of where to split.)
        """
        decision_rule = None
        best_info_gain = 0

        for feature in self.get_candidate_columns(X, self.rand):
            values = X[:, feature]
            sorted_indices = np.argsort(values)
            sorted_values = values[sorted_indices]
            for i in range(len(sorted_values) - 1):
                current_info_gain = self.information_gain(y[sorted_indices], np.arange(0,i+1), np.arange(i+1, len(values)))
                
                if(current_info_gain > best_info_gain):
                    split_value = self.midpoint(i, sorted_values)
                    best_info_gain = current_info_gain
                    decision_rule = (feature, split_value)

        return decision_rule
                
    def midpoint(self, index, y):
        """Finds the average value of entires at index i and i+1 in a presumably sorted array."""
        return (y[index] + y[index + 1])/2
    
    def information_gain(self, y , left_partition_indicies, right_partition_indicies):
        """
        Input: Takes an array of labels and the indicies of which belong to the lefr and right partition.
        Output: Returns information gain for this particular split.
        """
        n_left = len(left_partition_indicies)
        n_right = len(right_partition_indicies)
        n = n_left + n_right

        l_weight = n_left/n
        r_weight = n_right/n

        inf_gain = (self.gini_impurity(y) 
                    - self.gini_impurity(y[left_partition_indicies])*l_weight 
                    - self.gini_impurity(y[right_partition_indicies])*r_weight)

        return inf_gain

    def gini_impurity(self, y):
        """
        We can use this simplified version because we are solving a strictly binary classification problem, 
        assume y is a numpy array with values of 0 or 1.
        """

        label_one_probability = sum(y)/len(y)

        return 1 - ((label_one_probability)**2 + (1-label_one_probability)**2)


class TreeNode:
    
    def __init__(self, left, right, decision_rule):
        """Left and right are TreeNode objects. Decision rule is either a tuple with a feature and value 
        to split on or a single value which determines the leaf's predicted label.
        """
        self.left = left
        self.right = right
        self.decision_rule = decision_rule

    def predict(self, X):
        prediction = np.empty(len(X))
        
        if ((self.left is None) and (self.right is None)): # we are in a leaf node
            return self.decision_rule
        
        # get left and right indices
        left_i = np.where(X.T[self.decision_rule[0]] < self.decision_rule[1])
        right_i = np.where(X.T[self.decision_rule[0]] >= self.decision_rule[1])
        
        left_prediction = self.left.predict(X[left_i])
        right_prediction = self.right.predict(X[right_i])
        
        prediction[left_i] = left_prediction
        prediction[right_i] = right_prediction
               
        return prediction


class RandomForest:

    def __init__(self, rand=None, n=50):
        self.n = n
        self.rand = rand
        self.rftree = Tree(rand = rand, 
                           get_candidate_columns = random_sqrt_columns, 
                           min_samples = 2)  # initialize the tree properly

    def build(self, X, y):
        random_trees = []
        oob_list = []
        for i in range(self.n):
            bootstrap_indices = self.rand.choices(range(len(X)), k = len(X))
            out_of_bag_indices = np.setdiff1d(range(X.shape[0]), bootstrap_indices)
            #out_of_bag_indices = list(set(range(len(X))).difference(bootstrap_indices))
            
            random_trees.append(self.rftree.build(X[bootstrap_indices], y[bootstrap_indices]))
            oob_list.append(out_of_bag_indices)
        
        return RFModel(random_trees, oob_list, X, y, self.rand)


class RFModel:

    def __init__(self, tree_list, oob_list, X, y, rand):
        self.tree_list = tree_list
        self.oob_list = oob_list
        self.X = X
        self.y = y
        self.rand = rand

    def predict(self, X):
        predictions = np.zeros(len(X))
        for tree in self.tree_list:
            predictions += tree.predict(X)
        
        return (np.round(predictions/len(self.tree_list)))
    

    def importance(self):
        
        imps = np.zeros(self.X.shape[1])

        for i, tree in enumerate(self.tree_list):
            oob_indices = self.oob_list[i]
            X_oob = self.X[oob_indices].copy() # get an oob subset of X for predictions and shuffling
            baseline = misclassification_rate(tree.predict(X_oob),
                                              self.y[oob_indices])
            scores = np.zeros(self.X.shape[1])

            for j in range(self.X.shape[1]):
                temp = X_oob.copy()
                np.random.shuffle(temp[:,j]) # shuffle but without using the seed
                score = misclassification_rate(tree.predict(temp),
                                               self.y[oob_indices])

                scores[j] = score - baseline # feature score for j-th feature in i-th tree

            imps += scores

        return imps/len(self.tree_list)
    
def misclassification_rate(prediction, y):
    return np.mean(prediction != y)

def bootstrap(prediction, y, m=100):
    """Take an array of predictions and true values and return a bootstrap standard deviation."""
    bst = []
    for i in range(m):
        bootstrap_i = np.random.choice(range(len(prediction)),len(prediction))
        bst.append(misclassification_rate(prediction[bootstrap_i], y[bootstrap_i]))
    
    return(np.mean(bst), np.var(bst)) # return mean and variance of the bootstrap 

def hw_tree_full(train, test):
    """In function hw_tree_full, build a tree with min_samples=2.
    Return misclassification rates and standard errors (to quantify uncertainty) on training and testing data."""
    (X_train, y_train), (X_test, y_test) = train, test

    T = Tree(min_samples=2)
    tree = T.build(X_train, y_train)

    return (bootstrap(tree.predict(X_train), y_train), bootstrap(tree.predict(X_test), y_test))

def hw_randomforests(train, test):
    """In function hw_randomforest, use random forests with n=100 trees with min_samples=2. 
    Return misclassification rates and standard errors (to quantify uncertainty) on training and testing data."""
    (X_train, y_train), (X_test, y_test) = train, test

    F = RandomForest(rand = random.Random(3), n=100)
    rf = F.build(X_train, y_train)

    return(bootstrap(rf.predict(X_train), y_train), bootstrap(rf.predict(X_test), y_test))

def tki():
    train, test = get_data()
    legend = {"Bcr-abl":0, "Wild type":1}

    return train, test, legend
    
# this works
# if __name__ == "__main__":
#     learn, test, legend = tki()

#     print("full", hw_tree_full(learn, test))
#     print("random forests", hw_randomforests(learn, test))


In [15]:
hw_tree_full(train, test)

((0.0, 0.0), (0.18982758620689658, 0.003124227110582639))

In [11]:
hw_randomforests(train, test)

((0.0, 0.0), (0.03689655172413794, 0.0005827586206896552))

In [26]:
results = []
for i in [1,2,3,4,5,7,10,15,20,25,30,40,50,65,80,100]:
    F = RandomForest(rand=random.Random(3), n = i)
    forest = F.build(train[0],train[1])
    print((i, bootstrap(forest.predict(test[0]), test[1])))
    results.append((i, bootstrap(forest.predict(test[0]), test[1])))

(1, (0.25551724137931037, 0.00317705112960761))
(2, (0.17172413793103444, 0.0022646848989298454))
(3, (0.2022413793103448, 0.002650743162901308))
(4, (0.14017241379310344, 0.002185820451843044))
(5, (0.15413793103448278, 0.001925208085612366))
(7, (0.08965517241379312, 0.001474435196195006))
(10, (0.06965517241379313, 0.000915101070154578))
(15, (0.05413793103448276, 0.0008146254458977411))
(20, (0.05017241379310345, 0.0008745243757431628))
(25, (0.034482758620689655, 0.0005291319857312723))
(30, (0.03551724137931035, 0.0005994054696789536))
(40, (0.07689655172413792, 0.001155885850178359))
(50, (0.06672413793103447, 0.0008362366230677764))
(65, (0.03517241379310345, 0.0007070154577883473))
(80, (0.017241379310344827, 0.0002734839476813317))
(100, (0.03568965517241379, 0.0005722651605231867))


In [25]:
bootstrap(predictor.predict(test[0]), test[1])

(0.08310344827586208, 0.0011437574316290132)

In [ ]:
T = Tree(get_candidate_columns=random_sqrt_columns, rand = random.Random(0))

In [ ]:
pr = T.build(X1[:130], y1[:130])

In [ ]:
tree_prediction = pr.predict(X1[130:])

In [ ]:
misclassification_rate(tree_prediction, y1[130:])

In [ ]:
bootstrap(tree_prediction, y1[130:], 100)

In [17]:
RF = RandomForest(rand=random.Random(420), n = 20)

In [20]:
predictor = RF.build(train[0], train[1])

In [22]:
predictor.predict(test[0])

array([0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0.])

In [23]:
misclassification_rate(predictor.predict(test[0]), test[1])

0.08620689655172414

In [ ]:
# n = 50 Miscl.: 0.12068

In [ ]:
for i in range(50):
    RF = RandomForest(rand=random.Random(i), n = 100)
    predictor = RF.build(X1[:130],y1[:130])
    print(np.mean(predictor.predict(X1[130:]) != y1[130:]), i)

In [ ]:
imp = predictor.importance()

In [ ]:
max(imp)

In [ ]:
imp

In [ ]:
np.argsort(imp)